In [1]:
import pandas as pd
import numpy as np
import pickle
import Bio   
import plotly.express as px
import plotly.graph_objects as go
from jproperties import Properties
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, AutoModel, TrainingArguments, Trainer
from motif_utils import seq2kmer # Soruced from https://github.com/jerryji1993/DNABERT
import torch
from datasets import Dataset
import evaluate
import json
from load_data import load_data

c:\Users\zeusg\Envs\real-fast\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train, val, test = load_data(True, False, False)

In [5]:
all = pd.read_csv("cleaned/full_dataset.csv")

In [6]:
all

,Unnamed: 0,procedure_name,parameter_name,allele_accession_id,gene_accession_id,gene_symbol,allele_symbol,zygosity,strain,est_f_ea,...,p_m_ea,p_adj_m_ea,est_type_ea,add_info_ea,wt_f_ea_count,wt_m_ea_count,mut_f_ea_count,mut_m_ea_count,total_number_of_mutants,dna_seq
0,0,body composition (dexa lean/fat),body weight,mgi:5694170,MGI:1913300,0610009B22Rik,0610009B22Rik<tm1b(EUCOMM)Hmgu>,homozygote,c57bl/6n,-0.569790,...,0.003428,0.004570,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5034.0,5020.0,7.0,8.0,15.0,TAAAAG AAAAGG AAAGGA AAGGAA AGGAAT GGAATC GAAT...
1,1,body weight,body weight,not-released-0541dc0af7,MGI:1923511,0610040J01Rik,0610040J01Rik<em1(IMPC)Ccpcz>,homozygote,c57bl/6ncrl,-0.956256,...,0.086929,0.115905,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,8.0,8.0,16.0,CGACAG GACAGG ACAGGC CAGGCT AGGCTC GGCTCC GCTC...
2,2,body weight,body weight,mgi:5749856,MGI:1920971,1110017D15Rik,1110017D15Rik<em1(IMPC)H>,homozygote,c57bl/6ntac,-0.136771,...,0.423600,0.423600,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5807.0,5829.0,8.0,8.0,16.0,GCAGTC CAGTCT AGTCTT GTCTTT TCTTTG CTTTGT TTTG...
3,3,body weight,body weight,mgi:5603350,MGI:1915971,1110032A03Rik,1110032A03Rik<tm1.1(KOMP)Vlcg>,homozygote,c57bl/6ncrl,0.221307,...,0.181162,0.452904,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,8.0,8.0,16.0,TACTAG ACTAGA CTAGAC TAGACC AGACCA GACCAG ACCA...
4,4,body weight,body weight,mgi:4419470,MGI:1913452,1110059G10Rik,1110059G10Rik<tm1a(KOMP)Wtsi>,homozygote,c57bl/6n,0.016842,...,0.962555,0.989271,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,7.0,7.0,14.0,AATATT ATATTG TATTGT ATTGTT TTGTTG TGTTGG GTTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,5513,body weight,body weight,mgi:5792561,MGI:99182,Zscan21,Zscan21<tm2b(EUCOMM)Wtsi>,homozygote,c57bl/6n,-0.359866,...,0.096591,0.096591,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5724.0,5744.0,4.0,4.0,8.0,TCCAGA CCAGAG CAGAGG AGAGGG GAGGGG AGGGGC GGGG...
5493,5514,body weight,body weight,mgi:5905771,MGI:2159640,Zscan5b,Zscan5b<em1(IMPC)J>,homozygote,c57bl/6nj,0.111318,...,0.252846,0.637039,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",3415.0,3414.0,8.0,8.0,16.0,AAGCAC AGCACT GCACTT CACTTT ACTTTG CTTTGC TTTG...
5494,5515,body composition (dexa lean/fat),body weight,mgi:5700042,MGI:1919221,Zswim1,Zswim1<tm1b(KOMP)Mbp>,homozygote,c57bl/6n,-0.877491,...,0.000007,0.000014,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5034.0,5020.0,7.0,7.0,14.0,TTGACA TGACAG GACAGC ACAGCT CAGCTC AGCTCG GCTC...
5495,5516,body weight,body weight,mgi:5582436,MGI:1921714,Zswim5,Zswim5<tm1.1(KOMP)Wtsi>,homozygote,c57bl/6ncrl,0.211812,...,0.058867,0.074978,Overall genotype estimate,"""gender_included_in_analysis"":""Both sexes incl...",5083.0,5198.0,7.0,7.0,14.0,ATCTCT TCTCTC CTCTCC TCTCCA CTCCAG TCCAGC CCAG...


In [ ]:
with open("data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023.pkl", 'rb') as file:
        dna_dict =  pickle.load(file)
dna_dict

In [ ]:
df = pd.read_csv("D:/ML/bert_dna_weight/data/capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled.csv")
df

In [ ]:
1 - len(df.gene_symbol.unique())/len(df)


In [ ]:
df[df["gene_symbol"].duplicated()].head(10)

In [ ]:
train

In [ ]:
len(train.gene_symbol.unique()) + len(val.gene_symbol.unique()) + len(test.gene_symbol.unique())

In [ ]:

model = AutoModelForSequenceClassification.from_pretrained('zhihan1996/DNA_bert_6', 
                                                           num_labels=1, 
                                                           ignore_mismatched_sizes=True).to("cuda")
wandb.init(project="DNA-Weight", entity="pcoady")
wandb_config = wandb.config

In [ ]:
mse_metric = evaluate.load("mse")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    #labels = labels.reshape(-1, 1)
    
    wandb.log({'True Values': labels })
    wandb.log({'Predicted Values': logits})

    mse = mse_metric.compute(predictions=logits, references=labels)
    return mse

In [ ]:
training_args = TrainingArguments(output_dir='weight_model', 
                                  evaluation_strategy='epoch',
                                  per_device_train_batch_size = 5,  
                                  num_train_epochs=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=val,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.predict(test)
fig = go.Figure()
fig.add_trace(go.Histogram(x=results[1], histnorm='probability', name = "actual"))
fig.add_trace(go.Histogram(x=results[0].reshape(-1, 1)[:,0], histnorm='probability', name = "predictions"))

fig.update_layout(
    barmode="overlay",
    bargap=0.1)


fig.show()